Xin Xiong @ HKBU

## The free energy in the SK model by cavity method

- The total free energy: $F = \sum_{i} {\Delta F_i} + \sum_{a}{\Delta F_{a}} - \sum_{a}{|\partial a| \Delta F_{a}}$

- The free energy shift due to adding the function node $a$ is given by:
$$- \beta \Delta F_a = ln \frac{Z^{new}}{Z^{old}} = ln[cosh(\beta J_a) (1 + tanh(\beta J_a) \prod_{i \in \partial a} {m_{i \to a}})]$$.

- The free energy shift due to adding the variable node $i$ together with its neighboring function nodes $\{b \in \partial i\}$ is given by:
$$- \beta \Delta F_i = ln \frac{Z^{new}}{Z^{old}} = ln[\prod_{b \in \partial i}{\Lambda^{+}_{b \to i} + \prod_{b \in \partial i}{\Lambda^{-}_{b \to i}}}]$$
where, $\Lambda^{+}_{b \to i} = cosh(\beta J_b) (1 + tanh(\beta J_b) \prod_{j \in \partial b \backslash i} {m_{j \to b}})$, $\Lambda^{-}_{b \to i} = cosh(\beta J_b) (1 - tanh(\beta J_b) \prod_{j \in \partial b \backslash i} {m_{j \to b}})$.


In [1]:
import numpy as np
from scipy import stats
from itertools import combinations, product
import matplotlib.pyplot as plt

In [2]:
# define the number of variable nodes
n = 4
beta = 10
variable_nodes = ['i' + str(inx) for inx in range(n)]
variable_nodes

['i0', 'i1', 'i2', 'i3']

In [3]:
function_nodes = [f_node for f_node in combinations(variable_nodes, 2)]
function_nodes

[('i0', 'i1'),
 ('i0', 'i2'),
 ('i0', 'i3'),
 ('i1', 'i2'),
 ('i1', 'i3'),
 ('i2', 'i3')]

In [4]:
norm_dis = stats.norm(loc=0, scale=1/n)  # loc means mean, scale means variance

In [5]:
norm_dis.random_state = 42
interaction = norm_dis.rvs(len(function_nodes))
J = dict(zip(function_nodes, interaction))
J

{('i0', 'i1'): 0.12417853825280817,
 ('i0', 'i2'): -0.034566075292796164,
 ('i0', 'i3'): 0.16192213452517312,
 ('i1', 'i2'): 0.38075746410200634,
 ('i1', 'i3'): -0.05853834368083399,
 ('i2', 'i3'): -0.05853423923729514}

In [6]:
np.mean(interaction), np.std(interaction)

(0.08586991311151038, 0.1583357666583078)

### Calculate $h_{i \to a}$ and $u_{a \to i}$

In [7]:
def get_h_i2a(u_a_i, i2a, beta):
    i, a = i2a
    result_sum = 0
    for k, v in u_a_i.items():
        k_a, k_i = k
        if k_i == i and k_a != a:
            result_sum += beta * v
    return 1/beta * result_sum

def get_u_a2i(h_i_a, J, a2i, beta):
    a, i = a2i
    result_prod = 1
    for k, v in h_i_a.items():
        k_i, k_a = k
        if k_a == a and k_i != i:
            result_prod *= np.tanh(beta*v)
    return 1/beta * np.arctanh( np.tanh(beta * J[a]) * result_prod )

In [8]:
# initializing h_i2a, variable node -> function node
h_i2a = {}
for v_node in variable_nodes:
    for f_node in function_nodes:
        if v_node in f_node:
            h_i2a[(v_node, f_node)] = None
h_i2a

{('i0', ('i0', 'i1')): None,
 ('i0', ('i0', 'i2')): None,
 ('i0', ('i0', 'i3')): None,
 ('i1', ('i0', 'i1')): None,
 ('i1', ('i1', 'i2')): None,
 ('i1', ('i1', 'i3')): None,
 ('i2', ('i0', 'i2')): None,
 ('i2', ('i1', 'i2')): None,
 ('i2', ('i2', 'i3')): None,
 ('i3', ('i0', 'i3')): None,
 ('i3', ('i1', 'i3')): None,
 ('i3', ('i2', 'i3')): None}

In [9]:
# initializing u_b2i, function node -> variable node
u_a2i = {}
for f_node in function_nodes:
    for v_node in f_node:
        u_a2i[(f_node, v_node)] = np.random.random()
u_a2i

{(('i0', 'i1'), 'i0'): 0.9438512253354328,
 (('i0', 'i1'), 'i1'): 0.5618119492629121,
 (('i0', 'i2'), 'i0'): 0.8843671316723477,
 (('i0', 'i2'), 'i2'): 0.29487879092992964,
 (('i0', 'i3'), 'i0'): 0.5132819935662497,
 (('i0', 'i3'), 'i3'): 0.10050728604440706,
 (('i1', 'i2'), 'i1'): 0.8365160423233972,
 (('i1', 'i2'), 'i2'): 0.6938214629896227,
 (('i1', 'i3'), 'i1'): 0.14656520275272078,
 (('i1', 'i3'), 'i3'): 0.5671009397845732,
 (('i2', 'i3'), 'i2'): 0.9151477064930152,
 (('i2', 'i3'), 'i3'): 0.48611511791642026}

#### Solve $h$ and $u$ iteratively

In [10]:
delta_u_a2i_list = []
u_a2i_old = np.array(list(u_a2i.values()))
while 1:
    # print(u_a2i_old)
    for i2a in h_i2a.keys():
        h_i2a[i2a] = get_h_i2a(u_a_i=u_a2i, i2a=i2a, beta=beta)
    for a2i in u_a2i.keys():
        u_a2i[a2i] = get_u_a2i(h_i_a=h_i2a, J=J, a2i=a2i, beta=beta)
    u_a2i_new = np.array(list(u_a2i.values()))
    delta_u_a2i = np.sum(u_a2i_old - u_a2i_new)
    delta_u_a2i_list.append(delta_u_a2i)
    # print(delta_u_a2i)
    if np.abs(delta_u_a2i) < 1e-6:
        break
    u_a2i_old = u_a2i_new.copy()

In [11]:
u_a2i, h_i2a

({(('i0', 'i1'), 'i0'): 0.017760475132422977,
  (('i0', 'i1'), 'i1'): -0.01411901751294757,
  (('i0', 'i2'), 'i0'): -0.011776782592080872,
  (('i0', 'i2'), 'i2'): 0.003386240734264747,
  (('i0', 'i3'), 'i0'): -0.019220009821573743,
  (('i0', 'i3'), 'i3'): -0.008787756909824173,
  (('i1', 'i2'), 'i1'): 0.0179965805241015,
  (('i1', 'i2'), 'i2'): 0.01642698683500502,
  (('i1', 'i3'), 'i1'): 0.015618866804475166,
  (('i1', 'i3'), 'i3'): -0.007521400752717837,
  (('i2', 'i3'), 'i2'): 0.014512152406766267,
  (('i2', 'i3'), 'i3'): -0.014614523170320096},
 {('i0', ('i0', 'i1')): -0.016734308078550374,
  ('i0', ('i0', 'i2')): -0.010215010555148674,
  ('i0', ('i0', 'i3')): -0.009509469502177143,
  ('i1', ('i0', 'i1')): 0.021083299300090258,
  ('i1', ('i1', 'i2')): 0.016443482936061092,
  ('i1', ('i1', 'i3')): 0.014354755272029751,
  ('i2', ('i0', 'i2')): 0.03683384931500936,
  ('i2', ('i1', 'i2')): 0.018014717624945244,
  ('i2', ('i2', 'i3')): 0.028291373533284866,
  ('i3', ('i0', 'i3')): -0.02

In [12]:
len(delta_u_a2i_list)

7679

### Calculate $m_{i \to a}$
$m_{i \to a} = tanh(\beta h_{i \to a})$

In [13]:
def get_m_i2a(h_i2a, i2a, beta):
    return np.tanh(beta * h_i2a[i2a])

In [14]:
m_i2a = {}
for i2a, v in h_i2a.items():
    m_i2a[i2a] = get_m_i2a(h_i2a=h_i2a, i2a=i2a, beta=beta)
m_i2a

{('i0', ('i0', 'i1')): -0.16579830672399143,
 ('i0', ('i0', 'i2')): -0.10179628228854806,
 ('i0', ('i0', 'i3')): -0.09480908095920465,
 ('i1', ('i0', 'i1')): 0.20776367374670507,
 ('i1', ('i1', 'i2')): 0.16296864401282732,
 ('i1', ('i1', 'i3')): 0.14256963676506001,
 ('i2', ('i0', 'i2')): 0.3525375553322079,
 ('i2', ('i1', 'i2')): 0.17822337326048282,
 ('i2', ('i2', 'i3')): 0.27559965881452536,
 ('i3', ('i0', 'i3')): -0.2053710958935026,
 ('i3', ('i1', 'i3')): -0.29422810413629685,
 ('i3', ('i2', 'i3')): -0.27369609274850737}

### Calculate $\Delta F_a$
$$- \beta \Delta F_a = ln \frac{Z^{new}}{Z^{old}} = ln[cosh(\beta J_a) (1 + tanh(\beta J_a) \prod_{i \in \partial a} {m_{i \to a}})]$$

In [15]:
def base_f(J: dict, a: tuple, m_i2a: dict, symbol=1):
    """
    Calculate the free energy shif due to adding a function node a
    J: the interaction in each node pair
    a: the current function node, a
    m_i2a: the cavity magnetization of m_i when a was removed    
    """
    result_prod = 1
    for i in a:
        for k, v in m_i2a.items():
            k_i, k_a = k
            if k_i == i and k_a != a:
                result_prod *= v
    return np.cosh(beta * J[a]) * (1 + symbol * np.tanh(beta * J[a])*result_prod)   

In [16]:
def get_delta_f_a(J: dict, a: tuple, m_i2a: dict, beta=1):
    """
    Calculate the free energy shif due to adding a function node a
    J: the interaction in each node pair
    a: the current function node, a
    m_i2a: the cavity magnetization of m_i when a was removed
    beta: 
    """
    base_result = base_f(J=J, a=a, m_i2a=m_i2a, symbol=1)
    return - 1/beta * np.log(base_result)

### Calculate $\Delta F_i$
$$- \beta \Delta F_i = ln \frac{Z^{new}}{Z^{old}} = ln[\prod_{b \in \partial i}{\Lambda^{+}_{b \to i} + \prod_{b \in \partial i}{\Lambda^{-}_{b \to i}}}]$$
where, $\Lambda^{+}_{b \to i} = cosh(\beta J_b) (1 + tanh(\beta J_b) \prod_{j \in \partial b \backslash i} {m_{j \to b}})$, $\Lambda^{-}_{b \to i} = cosh(\beta J_b) (1 - tanh(\beta J_b) \prod_{j \in \partial b \backslash i} {m_{j \to b}})$


In [17]:
def get_delta_f_i(J: dict, i: str, m_i2a: dict, function_nodes: list, beta=1):
    """
    
    """
    a_of_i = [a for a in function_nodes if i in a]
    prod_part_plus = 1
    prod_part_minus = 1
    for a in a_of_i:
        prod_part_plus *= base_f(J=J, a=a, m_i2a=m_i2a, symbol=1)
        prod_part_minus *= base_f(J=J, a=a, m_i2a=m_i2a, symbol=-1)
    return - 1/beta * np.log(prod_part_plus + prod_part_minus)

### Calculate the total free energy

 $F = \sum_{i} {\Delta F_i} + \sum_{a}{\Delta F_{a}} - \sum_{a}{|\partial a| \Delta F_{a}}$

In [18]:
f = 0
sum_f_a = 0
sum_f_i = 0
sum_last_item = 0
for i in variable_nodes:
    sum_f_i += get_delta_f_i(J=J, i=i, m_i2a=m_i2a, function_nodes=function_nodes, beta=beta)
for a in function_nodes:
    delta_f_a = get_delta_f_a(J=J, a=a, m_i2a=m_i2a, beta=beta)
    sum_f_a += delta_f_a
    sum_last_item += len(a) * delta_f_a
# print(sum_f_a, sum_f_i, sum_last_item)
f = sum_f_a + sum_f_i - sum_last_item
f

-0.786306549007121

### Calculate $m_i$
- $m_i = tanh(\sum_{b \in \partial i}{\beta u_{b \to i}})$

In [19]:
def get_m_i(beta, u_a_i, i):
    result_sum = 0
    for k, v in u_a_i.items():
        k_a, k_i = k
        if i in k_a and k_i != i:  # the function node of i, but exclude i
            result_sum += beta * v
    return np.tanh(result_sum)

In [20]:
m_i = {}
for i in variable_nodes:
    m_i[i] = get_m_i(beta=beta, u_a_i=u_a2i, i=i)
m_i

{'i0': -0.19276311347217465,
 'i1': 0.2605148022327441,
 'i2': -0.08375061062281816,
 'i3': 0.10867916020828047}

## Calculate the free energy by numerical enumeration
- $F = -\frac{1}{\beta}lnZ$
- $Z = \sum_{\sigma}{e^{-\beta H(\sigma)}}$
- $H_N(\sigma) = \frac{1}{\sqrt{N}} \sum_{1 \leq i < j \leq N}{J_{ij}\sigma_i \sigma_j}$

In [21]:
def get_h_sigma(n: int, sigma_dict: dict, J: dict):
    """
    Calculate H_sigma for a single configuraion
    n: the number of variable nodes (spins)
    sigma_dict: sigma with key (variable name)
    J: interaction, { function_node: f_interaction }
    """
    h_sigma = 0
    for k, v in J.items():
        i, j = k
        h_sigma += J[k] * sigma_dict[i] * sigma_dict[j]
    return 1/np.sqrt(n) * h_sigma

def get_z(beta: float, n: int):
    """
    beta: 
    sigma: a list of all configurations, {-1, +1}^N
    """
    if n > 16:
        raise ValueError('Too big n')
    norm_dis = stats.norm(loc=0, scale=1)
    variable_nodes = ['i' + str(inx) for inx in range(n)]
    function_nodes = [f_node for f_node in combinations(variable_nodes, 2)]
    
    spins = [[-1, 1]] * n
    all_sigma = []
    for element in product(*spins):
        all_sigma.append(element)
    print(f'Current n is {n}, there have {np.power(2, n)} items in the total configurations')
    z_sum = 0
    for sigma in all_sigma:
        sigma_dict = dict(zip(variable_nodes, sigma))
        interaction = norm_dis.rvs(len(function_nodes))
        J = dict(zip(function_nodes, interaction))
        h_sigma = get_h_sigma(n=n, sigma_dict=sigma_dict, J=J)
        z_sum += np.power(np.e, -1 * beta * h_sigma)
    # print(h_sigma_list)
    return z_sum

def get_f_by_z(z, beta):
    return -1/beta * np.log(z)

In [22]:
# %%timeit
z = get_z(beta=beta, n=n)

Current n is 4, there have 16 items in the total configurations


In [23]:
z

3218235004.0983906

In [24]:
f = get_f_by_z(z=z, beta=beta)

In [25]:
f

-2.1892098910848445